## 변수와 참조변수를 런타입에 처리하기 

- Cell 은 불변변수와 갸변변수를 처리, 단 Cell의 소유권 이동은 불가능하다, 
- RefCell 은 불변참조와 가변참조를 처리 

## 1. Cell 타입 

- Cell 타입은 값의 이동에 따른 내부 가변성을 구현합니다. 
- 단일 스레드 환경에서 데이터 경쟁을 방지하기 위해 사용되는 타입으로, 소유권 이동을 지원하지 않습니다.

### Cell 타입은 

- new로 생성하다. 
- 내부의 값은 get() 메서드

In [30]:
use std::cell::Cell;

fn main() {
    let cell = Cell::new(1);

    println!("before : {}", cell.get());

    println!("after : {}", cell.get());
}

In [31]:
main();

before : 1
after : 1


### 내부의 값을 변경하기

- set 메서드로 값을 변경 , 새로운 소유권을 만듬

In [28]:
use std::cell::Cell;

fn main() {
    let cell = Cell::new(1);

    println!("before : {}", cell.get());

    cell.set(2);

    println!("after : {}", cell.get());
}

In [29]:
main();

before : 1
after : 2


### Cell 인스턴스의 소유권 이동은 불가

In [39]:
fn main() {
    let cell1 = Cell::new("문자열".to_string());
    let cell2 = cell1;         // 소유권 이전 시도

    println!("cell2 value: {}", cell2.get());
}

Error: the method `get` exists for struct `Cell<String>`, but its trait bounds were not satisfied

### Copy 트레이트 처리일 때는 값이 복사 

In [41]:
fn main() {
    let cell1 = Cell::new("문자열");
    let cell2 = cell1;                // 값이 복사되어 처리된다. 

    println!("cell2 value: {}", cell2.get());
}

In [42]:
main();

cell2 value: 문자열



### Cell<T> 타입은 다음과 같은 메소드를 제공합니다.

- get: 내부 데이터의 불변 참조를 반환합니다.
- set: 내부 데이터를 새로운 값으로 설정합니다.
- replace: 내부 데이터를 새로운 값으로 설정하고 이전 값을 반환합니다.
- take: 내부 데이터를 가져와서 새로운 값으로 설정하고 이전 값을 반환합니다.

In [32]:
fn main() {
    let data = Cell::new(42);
    let data_value = data.get();
    println!("data_value: {}", data_value);

    data.set(100);
    let new_data_value = data.get();
    println!("new_data_value: {}", new_data_value);

    let old_data_value = data.replace(200);
    println!("old_data_value: {}", old_data_value);
    println!("new_data_value: {}", data.get());

    let taken_value = data.take();
    println!("taken_value: {}", taken_value);
    println!("new_data_value: {}", data.get());
}

In [33]:
main();

data_value: 42
new_data_value: 100
old_data_value: 100
new_data_value: 200
taken_value: 200
new_data_value: 0


### Cell<T> 단일 값만 처리

- Cell<T>을 사용하여 벡터를 처리하려면, 각각의 요소에 대해 Cell<T> 값을 생성하고, 이를 담은 벡터를 사용해야 합니다.

In [43]:
fn main() {
    let vec = vec![1, 2, 3, 4, 5];
    let vec_with_cells: Vec<Cell<i32>> = vec.iter().map(|&x| Cell::new(x)).collect();

    for cell in vec_with_cells.iter() {
        let value = cell.get();
        println!("value: {}", value);
        cell.set(value * 2);
    }

    let vec_with_doubled_values: Vec<i32> = vec_with_cells.iter().map(|cell| cell.get()).collect();
    println!("vec_with_doubled_values: {:?}", vec_with_doubled_values);
}

In [44]:
main();

value: 1
value: 2
value: 3
value: 4
value: 5
vec_with_doubled_values: [2, 4, 6, 8, 10]


## 2. RefCell<T> 타입

- Rust에서는 기본적으로 런타임에서 발생할 수 있는 데이터 경합(데이터 경쟁)을 방지하기 위해 여러 가지 방법을 제공합니다. 
    
- RefCell<T> 타입은 런타임에 여러 개의 가변 참조를 제공하면서도 데이터 경합을 방지하는 기능을 제공합니다. 
- 이를 가능하게 하는 방법은, RefCell<T> 내부에서 참조가 가변인지 불변인지를 추적하면서 불변 참조가 있을 때는 가변 참조를 허용하지 않고, 
- 가변 참조가 있을 때는 불변 참조를 허용하지 않는 것입니다. 
- 이를 통해 런타임에서 데이터 경합을 방지할 수 있습니다.
    
- RefCell<T>는 기본적으로 스레드 간 동기화를 지원하지 않기 때문에, 멀티스레드 환경에서는 Arc<Mutex<T>>나 Arc<RwLock<T>> 등과 같은 다른 타입을 사용해야 합니다.


## 참조 접근 처리 

- Ref<T>와 RefMut<T>는 빌림(borrowing)을 사용하여, 참조자를 안전하게 사용하는 방법 중 하나입니다. 
- 이 두 타입은 std::cell::RefCell<T> 타입에서 생성됩니다.

### 참조자일 경우 정수도 처리 가능 

In [45]:
use std::cell::RefCell;

In [46]:

fn main() {
    let x = RefCell::new(5);
    let mut y = x.borrow_mut();
    *y += 1;
    println!("{}", y);
}

In [47]:
main();

6


### 'Ref<T> 

- 불변 참조자로, RefCell<T>의 값을 참조하기 위해 사용됩니다. 
- 이를 사용하면 런타임에서 불변 참조자가 다수일 경우에도 안전하게 T 값을 공유할 수 있습니다. 
- 이는 다른 참조자 또는 &mut T 참조자와 함께 사용될 수 없습니다.

In [10]:
use std::cell::RefCell;

fn main() {
    let shared_data = RefCell::new(vec![1, 2, 3]);
    let data_ref = shared_data.borrow();
    println!("data_ref: {:?}", data_ref);
    
    
    let data_ref_2 = shared_data.borrow();
    println!("data_ref_2: {:?}", data_ref_2);

}

In [11]:
main();

data_ref: [1, 2, 3]
data_ref_2: [1, 2, 3]


In [ ]:
### 참조 스마트 포인터 타입 확인 

In [18]:
use std::cell::{RefCell, Ref};
fn main() {
    let data = RefCell::new(vec![1, 2, 3]);
    let data_ref = data.borrow();
    println!("Type of data_ref: {:?}", std::any::type_name::<Ref<Vec<i32>>>());
    println!("data_ref: {:?}", data_ref);
}

In [19]:
main();

Type of data_ref: "core::cell::Ref<alloc::vec::Vec<i32>>"
data_ref: [1, 2, 3]


### 'RefMut<T>
- 가변 참조자로, RefCell<T>의 값을 변경하기 위해 사용됩니다. 
- RefCell<T>을 런타임에서 변경할 수 있는 일종의 "가변 참조자"로 생각할 수 있습니다. 
- RefMut<T>는 불변 참조자와 다르게 동시에 여러 개를 생성할 수 없으며, &T 참조자와 함께 사용할 수 없습니다.

In [12]:
fn main() {
    let shared_data = RefCell::new(vec![1, 2, 3]);
    let mut data_ref_mut = shared_data.borrow_mut();
    data_ref_mut.push(4);
    println!("data_ref_mut: {:?}", data_ref_mut);
    
    let data_ref_3 = shared_data.borrow();
    println!("data_ref_3: {:?}", data_ref_3);
    
}

In [13]:
main();

data_ref_mut: [1, 2, 3, 4]


thread '<unnamed>' panicked at 'already mutably borrowed: BorrowError', src/lib.rs:9:34
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


### 가변 참조는 항상 1번만 사용해야 한다

In [14]:
fn main() {
    let shared_data = RefCell::new(vec![1, 2, 3]);
    let mut data_ref_mut = shared_data.borrow_mut();
    data_ref_mut.push(4);
    println!("data_ref_mut: {:?}", data_ref_mut);
    
    //let data_ref_3 = shared_data.borrow();
    // println!("data_ref_3: {:?}", data_ref_3);
    
}

In [15]:
main();

data_ref_mut: [1, 2, 3, 4]


### 가변참조 스마트 포인터 타입 확인 

In [25]:
use std::cell::{RefCell, RefMut};
fn main() {
    let data = RefCell::new(vec![1, 2, 3]);
    let mut data_ref_mut = data.borrow_mut();
    println!("Type of data_ref_mut: {:?}", std::any::type_name::<RefMut<Vec<i32>>>());
    data_ref_mut.push(4);
    println!("data_ref_mut: {:?}", data_ref_mut);
}

In [26]:
main();

Type of data_ref_mut: "core::cell::RefMut<alloc::vec::Vec<i32>>"
data_ref_mut: [1, 2, 3, 4]
